In [ ]:
import syft as sy
import numpy as np
from syft.lib.adp.tensor import Tensor
from syft.lib.adp.entity import Entity
from syft.lib.adp.adversarial_accountant import AdversarialAccountant

entities = [Entity(name="Tudor"), Entity(name="Madhava"), Entity(name="Kritika"), Entity(name="George")]

x = Tensor(np.array([[1,1],[1,0],[0,1],[0,0]])).private(min_val=0, max_val=1, entities=entities)
y = Tensor(np.array([[1],[1],[0],[0]])).private(min_val=0, max_val=1, entities=entities)

acc = AdversarialAccountant(max_budget=3000)

weights = Tensor(np.random.uniform(size=(2,1)))

for i in range(10):
    
    pred = x.dot(weights)
    loss = np.mean(np.square(y-pred))
    loss.backward()

    weight_grad = (weights.grad * 1)
    print(weight_grad)
    
    weight_grad = weight_grad.publish(acc=acc, sigma=0.1)
    
    weights = weights - weight_grad
    print(loss.value)
    acc.print_ledger()

[[2*NpnFM_George*(-0.25*CQBZG_George + 0.174332827299456*NpnFM_George + 0.00653390754098843*iIRfL_George) + 2*XAIsx_Madhava*(0.00653390754098843*LyjPI_Madhava + 0.174332827299456*XAIsx_Madhava - 0.25*tiIUF_Madhava) + 2*XOkwG_Kritika*(0.00653390754098843*LAgRZ_Kritika + 0.174332827299456*XOkwG_Kritika - 0.25*oUvCe_Kritika) + 2*tTPzN_Tudor*(-0.25*YXQNa_Tudor + 0.00653390754098843*hmoIH_Tudor + 0.174332827299456*tTPzN_Tudor)=-0.28960087572019844]
 [2*LAgRZ_Kritika*(0.00653390754098843*LAgRZ_Kritika + 0.174332827299456*XOkwG_Kritika - 0.25*oUvCe_Kritika) + 2*LyjPI_Madhava*(0.00653390754098843*LyjPI_Madhava + 0.174332827299456*XAIsx_Madhava - 0.25*tiIUF_Madhava) + 2*hmoIH_Tudor*(-0.25*YXQNa_Tudor + 0.00653390754098843*hmoIH_Tudor + 0.174332827299456*tTPzN_Tudor) + 2*iIRfL_George*(-0.25*CQBZG_George + 0.174332827299456*NpnFM_George + 0.00653390754098843*iIRfL_George)=-0.12519871523713394]]
beginning an epic search...


[[2*OWKcD_Kritika*(0.245248030714688*HoJmI_Kritika + 0.0517686228800157*OWKcD_Kritika - 0.25*PodKD_Kritika) + 2*fEvNh_Tudor*(-0.25*RkusQ_Tudor + 0.0517686228800157*fEvNh_Tudor + 0.245248030714688*oNSSs_Tudor) + 2*mIvyq_George*(0.245248030714688*JNWok_George - 0.25*OMYpO_George + 0.0517686228800157*mIvyq_George) + 2*rDFEx_Madhava*(-0.25*Gobpc_Madhava + 0.245248030714688*fbfkR_Madhava + 0.0517686228800157*rDFEx_Madhava)=-0.3024294470505602]
 [2*HoJmI_Kritika*(0.245248030714688*HoJmI_Kritika + 0.0517686228800157*OWKcD_Kritika - 0.25*PodKD_Kritika) + 2*JNWok_George*(0.245248030714688*JNWok_George - 0.25*OMYpO_George + 0.0517686228800157*mIvyq_George) + 2*fbfkR_Madhava*(-0.25*Gobpc_Madhava + 0.245248030714688*fbfkR_Madhava + 0.0517686228800157*rDFEx_Madhava) + 2*oNSSs_Tudor*(-0.25*RkusQ_Tudor + 0.0517686228800157*fEvNh_Tudor + 0.245248030714688*oNSSs_Tudor)=0.5845293686187853]]
beginning an epic search...


KeyboardInterrupt: 